In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications import VGG19
from keras.models import Model
from keras.layers.core import Activation
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

import tensorflow as tf
import random
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


labels.csv
sample_submission.csv
test
train



In [2]:
##############
# parameters #
##############
# dontFreezeLast = 0;

# patience = 10;

# gpuName = '/device:GPU:0'
# workers = 2;
# histogram_freq = 0;

# epochs = 100;
# validation_size=0.3;

modelPath = '../models/VGG19_opt/';
modelName = 'run4.h5';
os.environ["CUDA_VISIBLE_DEVICES"]="0"

if not os.path.exists(modelPath):
    os.makedirs(modelPath)

First we will read in the csv's so we can see some more information on the filenames and breeds

In [3]:
# df_train = pd.read_csv('../input/labels.csv')
# df_test = pd.read_csv('../input/sample_submission.csv')

# print('Training images: ',df_train.shape[0])
# print('Test images: ',df_test.shape[0])

# reduce dimensionality
#df_train = df_train.head(100)
#df_test = df_test.head(100)

In [4]:
#df_train.head(10)

We can see that the breed needs to be one-hot encoded for the final submission, so we will now do this.

In [5]:
# targets_series = pd.Series(df_train['breed'])
# one_hot = pd.get_dummies(targets_series, sparse = True)

In [6]:
# one_hot_labels = np.asarray(one_hot)

Next we will read in all of the images for test and train, using a for loop through the values of the csv files. I have also set an im_size variable which sets the size for the image to be re-sized to, 90x90 px, you should play with this number to see how it affects accuracy.

In [7]:
# im_size = 90

In [8]:
# x_train = []
# y_train = []
# x_test = []

In [9]:
# i = 0 
# for f, breed in tqdm(df_train.values[:10]):
#     img = cv2.imread('../input/train/{}.jpg'.format(f))
#     label = one_hot_labels[i]
#     x_train.append(cv2.resize(img, (im_size, im_size)))
#     y_train.append(label)
#     i += 1

In [10]:
# for f in tqdm(df_test['id'].values):
#     img = cv2.imread('../input/test/{}.jpg'.format(f))
#     x_test.append(cv2.resize(img, (im_size, im_size)))

In [11]:
# y_train_raw = np.array(y_train, np.uint8)
# x_train_raw = np.array(x_train, np.float32) / 255.
# x_test  = np.array(x_test, np.float32) / 255.

We check the shape of the outputs to make sure everyting went as expected.

In [12]:
# print(x_train_raw.shape)
# print(y_train_raw.shape)
# print(x_test.shape)

We can see above that there are 120 different breeds. We can put this in a num_class variable below that can then be used when creating the CNN model.

In [13]:
# num_class = y_train_raw.shape[1]
# print('Number of classes: ', num_class)

It is important to create a validation set so that you can gauge the performance of your model on independent data, unseen to the model in training. We do this by splitting the current training set (x_train_raw) and the corresponding labels (y_train_raw) so that we set aside 30 % of the data at random and put these in validation sets (X_valid and Y_valid).

* This split needs to be improved so that it contains images from every class, with 120 separate classes some can not be represented and so the validation score is not informative.

In [14]:
# X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=validation_size, random_state=1)

Now we build the CNN architecture. Here we are using a pre-trained model VGG19 which has already been trained to identify many different dog breeds (as well as a lot of other objects from the imagenet dataset see here for more information: http://image-net.org/about-overview). Unfortunately it doesn't seem possible to downlod the weights from within this kernel so make sure you set the weights argument to 'imagenet' and not None, as it currently is below.

We then remove the final layer and instead replace it with a single dense layer with the number of nodes corresponding to the number of breed classes we have (120).

In [15]:
def data():
    print('Getting data')
    df_train = pd.read_csv('../input/labels.csv')
    df_test = pd.read_csv('../input/sample_submission.csv')
    
    targets_series = pd.Series(df_train['breed'])
    one_hot = pd.get_dummies(targets_series, sparse = True)
    one_hot_labels = np.asarray(one_hot)
    
    im_size = 90;
    x_train = []
    y_train = []
    x_test = []
    
    i = 0 
    for f, breed in tqdm(df_train.values):
        img = cv2.imread('../input/train/{}.jpg'.format(f))
        label = one_hot_labels[i]
        x_train.append(cv2.resize(img, (im_size, im_size)))
        y_train.append(label)
        i += 1
    
    y_train_raw = np.array(y_train, np.uint8)
    x_train_raw = np.array(x_train, np.float32) / 255.
    num_class = y_train_raw.shape[1]
    
    print('Splitting into training/validation')
    X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.3, random_state=1)
    
    return X_train, Y_train, X_valid, Y_valid

In [16]:
# prepare data and model for hyperas

def model(X_train,Y_train,X_valid,Y_valid):
    print('Creating model')
    base_model = VGG19(weights = 'imagenet',
                       include_top=False,
                       input_shape=(im_size, im_size, 3))
    
    dropout=0.6;
    dontFreeze = 3;
    batchSize = 64;
    momentum=0.99;
    #
    lambda_l2 = {{uniform(0.000001,0.1)}};
    bottleneckFlag = {{choice([True,False])}};
    
    print()
    print('lambda_l2=',lambda_l2)
    print('bottleneckFlag=',bottleneckFlag)
    print()
    
    stepsPerEpoch = round( len(X_train)/batchSize );
    
    # Add a new top layer
    x = base_model.output
    x = BatchNormalization(axis=1,momentum=momentum)(x)
    x = Flatten()(x)
    x = Dropout(dropout)(x)
    
    x = Dense(2048,activation='relu',kernel_regularizer=l2(lambda_l2))(x)
    if bottleneckFlag:
        x = Dense(100,activation='relu')(x)
    x = Dense(1024,activation='relu')(x)
    x = Dense(512,activation='relu')(x)
    # in any case:
    predictions = Dense(num_class, activation='softmax')(x)

    # This is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    # First: train only the top layers (which were randomly initialized)
    for i in range(len(base_model.layers)-dontFreeze):
        base_model.layers[i].trainable = False

    # predetermined optimizer
    lr=0.00020389590556056983;
    beta_1=0.9453158868247398;
    beta_2=0.9925872692991417;
    decay=0.000821336141287018;
    adam = keras.optimizers.Adam(lr=lr,beta_1=beta_1,beta_2=beta_2,decay=decay)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=adam, 
                  metrics=['accuracy'])

    callbacks_list = [];
    callbacks_list.append(keras.callbacks.EarlyStopping(
        monitor='val_acc',
        patience=10,
        verbose=1));


    # data augmentation & fitting
    datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.5,
        zoom_range=0.5,
        horizontal_flip=True,
        vertical_flip=True);
    
    model.fit_generator(
        datagen.flow(X_train,Y_train,batch_size=batchSize),
        steps_per_epoch=stepsPerEpoch,
        epochs=100,
        verbose=1,
        validation_data=(X_valid,Y_valid),
        workers=2,
        shuffle=True,
        callbacks=callbacks_list)
#     model.fit(X_train, Y_train,
#       epochs=100,
#       batch_size = batchSize,
#       validation_data=(X_valid, Y_valid),
#       verbose=1,
#       callbacks=callbacks_list)

    score, acc = model.evaluate(X_valid, Y_valid, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}


In [17]:
best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='VGG19Optimization_4')

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras.applications import VGG19
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.layers.core import Activation
except:
    pass

try:
    from keras.layers import Dense, Dropout, Flatten, BatchNormalization
except:
    pass

try:
    from keras.regularizers import l2
except:
    pass

try:
    from keras.preprocessing.image import ImageDataGenerator
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import random
except:
    pass

try:
    import os
except:
    pass

try:
    from tqdm import tqdm
except:
    pass

try:
    from sklearn import 

100%|██████████| 10222/10222 [00:43<00:00, 235.31it/s]


Splitting into training/validation
Creating model

lambda_l2= 0.04371218882692479
bottleneckFlag= True

Epoch 1/100
112/112 [==============================] - 37s 332ms/step - loss: 45.6191 - acc: 0.0090 - val_loss: 16.4707 - val_acc: 0.0072
Epoch 2/100
112/112 [==============================] - 31s 281ms/step - loss: 9.1396 - acc: 0.0102 - val_loss: 5.6122 - val_acc: 0.0104
Epoch 3/100
112/112 [==============================] - 32s 282ms/step - loss: 5.0461 - acc: 0.0092 - val_loss: 4.8379 - val_acc: 0.0104
Epoch 4/100
112/112 [==============================] - 31s 280ms/step - loss: 4.8018 - acc: 0.0118 - val_loss: 4.7780 - val_acc: 0.0137
Epoch 5/100
112/112 [==============================] - 32s 282ms/step - loss: 4.7545 - acc: 0.0142 - val_loss: 4.6931 - val_acc: 0.0176
Epoch 6/100
112/112 [==============================] - 31s 275ms/step - loss: 4.7071 - acc: 0.0159 - val_loss: 4.6526 - val_acc: 0.0209
Epoch 7/100
112/112 [==============================] - 32s 286ms/step - loss: 

112/112 [==============================] - 32s 286ms/step - loss: 3.7618 - acc: 0.1205 - val_loss: 3.7585 - val_acc: 0.1278
Epoch 60/100
112/112 [==============================] - 32s 284ms/step - loss: 3.7765 - acc: 0.1232 - val_loss: 3.7664 - val_acc: 0.1291
Epoch 61/100
112/112 [==============================] - 31s 276ms/step - loss: 3.7510 - acc: 0.1273 - val_loss: 3.7278 - val_acc: 0.1314
Epoch 62/100
112/112 [==============================] - 31s 276ms/step - loss: 3.7442 - acc: 0.1278 - val_loss: 3.7603 - val_acc: 0.1321
Epoch 63/100
112/112 [==============================] - 30s 272ms/step - loss: 3.7458 - acc: 0.1280 - val_loss: 3.7283 - val_acc: 0.1373
Epoch 64/100
112/112 [==============================] - 31s 274ms/step - loss: 3.7348 - acc: 0.1245 - val_loss: 3.7505 - val_acc: 0.1314
Epoch 65/100
112/112 [==============================] - 31s 281ms/step - loss: 3.7256 - acc: 0.1327 - val_loss: 3.7400 - val_acc: 0.1275
Epoch 66/100
112/112 [==============================] 

Epoch 18/100
112/112 [==============================] - 31s 273ms/step - loss: 4.0096 - acc: 0.1109 - val_loss: 3.9176 - val_acc: 0.1164
Epoch 19/100
112/112 [==============================] - 32s 288ms/step - loss: 3.9807 - acc: 0.1146 - val_loss: 3.9067 - val_acc: 0.1239
Epoch 20/100
112/112 [==============================] - 31s 272ms/step - loss: 3.9617 - acc: 0.1150 - val_loss: 3.8531 - val_acc: 0.1226
Epoch 21/100
112/112 [==============================] - 32s 287ms/step - loss: 3.9302 - acc: 0.1183 - val_loss: 3.8732 - val_acc: 0.1262
Epoch 22/100
112/112 [==============================] - 32s 286ms/step - loss: 3.9082 - acc: 0.1258 - val_loss: 3.8140 - val_acc: 0.1396
Epoch 23/100
112/112 [==============================] - 32s 289ms/step - loss: 3.8816 - acc: 0.1266 - val_loss: 3.8864 - val_acc: 0.1265
Epoch 24/100
112/112 [==============================] - 31s 276ms/step - loss: 3.8808 - acc: 0.1253 - val_loss: 3.8177 - val_acc: 0.1363
Epoch 25/100
112/112 [===================

...

112/112 [==============================] - 31s 273ms/step - loss: 4.5536 - acc: 0.0274 - val_loss: 4.4586 - val_acc: 0.0339
Epoch 9/100
112/112 [==============================] - 31s 273ms/step - loss: 4.4956 - acc: 0.0306 - val_loss: 4.4139 - val_acc: 0.0267
Epoch 10/100
112/112 [==============================] - 30s 272ms/step - loss: 4.4663 - acc: 0.0313 - val_loss: 4.3436 - val_acc: 0.0404
Epoch 11/100
112/112 [==============================] - 31s 275ms/step - loss: 4.4197 - acc: 0.0409 - val_loss: 4.3089 - val_acc: 0.0408
Epoch 12/100
112/112 [==============================] - 31s 272ms/step - loss: 4.4047 - acc: 0.0411 - val_loss: 4.2653 - val_acc: 0.0496
Epoch 13/100
112/112 [==============================] - 31s 273ms/step - loss: 4.3724 - acc: 0.0444 - val_loss: 4.2474 - val_acc: 0.0515
Epoch 14/100
112/112 [==============================] - 30s 272ms/step - loss: 4.3423 - acc: 0.0481 - val_loss: 4.1699 - val_acc: 0.0574
Epoch 15/100
112/112 [==============================] -

112/112 [==============================] - 31s 273ms/step - loss: 3.6278 - acc: 0.1460 - val_loss: 3.7473 - val_acc: 0.1448
Epoch 00067: early stopping
Test accuracy: 0.144766873168
Creating model

lambda_l2= 0.0019418805465617039
bottleneckFlag= False

Epoch 1/100
112/112 [==============================] - 35s 313ms/step - loss: 8.4531 - acc: 0.0087 - val_loss: 7.9638 - val_acc: 0.0111
Epoch 2/100
112/112 [==============================] - 31s 279ms/step - loss: 7.5419 - acc: 0.0099 - val_loss: 7.1235 - val_acc: 0.0095
Epoch 3/100
112/112 [==============================] - 31s 274ms/step - loss: 6.7692 - acc: 0.0109 - val_loss: 6.4250 - val_acc: 0.0143
Epoch 4/100
112/112 [==============================] - 31s 276ms/step - loss: 6.1528 - acc: 0.0157 - val_loss: 5.8766 - val_acc: 0.0202
Epoch 5/100
112/112 [==============================] - 31s 274ms/step - loss: 5.6647 - acc: 0.0253 - val_loss: 5.4244 - val_acc: 0.0333
Epoch 6/100
112/112 [==============================] - 31s 274ms/s

112/112 [==============================] - 31s 275ms/step - loss: 3.2941 - acc: 0.2410 - val_loss: 3.6675 - val_acc: 0.1940
Epoch 60/100
112/112 [==============================] - 31s 274ms/step - loss: 3.2918 - acc: 0.2389 - val_loss: 3.6516 - val_acc: 0.1927
Epoch 61/100
112/112 [==============================] - 31s 273ms/step - loss: 3.2550 - acc: 0.2371 - val_loss: 3.6137 - val_acc: 0.2044
Epoch 62/100
112/112 [==============================] - 31s 281ms/step - loss: 3.2355 - acc: 0.2497 - val_loss: 3.6262 - val_acc: 0.2018
Epoch 63/100
112/112 [==============================] - 31s 277ms/step - loss: 3.2438 - acc: 0.2513 - val_loss: 3.6271 - val_acc: 0.1976
Epoch 64/100
112/112 [==============================] - 31s 275ms/step - loss: 3.2218 - acc: 0.2491 - val_loss: 3.6139 - val_acc: 0.1986
Epoch 65/100
112/112 [==============================] - 32s 287ms/step - loss: 3.2130 - acc: 0.2453 - val_loss: 3.6276 - val_acc: 0.1914
Epoch 66/100
112/112 [==============================] 

Epoch 47/100
112/112 [==============================] - 31s 280ms/step - loss: 3.5023 - acc: 0.1962 - val_loss: 3.5845 - val_acc: 0.1875
Epoch 48/100
112/112 [==============================] - 31s 280ms/step - loss: 3.4869 - acc: 0.1974 - val_loss: 3.5854 - val_acc: 0.1849
Epoch 49/100
112/112 [==============================] - 31s 275ms/step - loss: 3.4784 - acc: 0.1925 - val_loss: 3.6058 - val_acc: 0.1855
Epoch 50/100
112/112 [==============================] - 31s 275ms/step - loss: 3.4339 - acc: 0.2024 - val_loss: 3.5886 - val_acc: 0.1979
Epoch 51/100
112/112 [==============================] - 31s 274ms/step - loss: 3.4625 - acc: 0.1975 - val_loss: 3.5917 - val_acc: 0.1901
Epoch 52/100
112/112 [==============================] - 31s 279ms/step - loss: 3.4554 - acc: 0.1971 - val_loss: 3.6155 - val_acc: 0.1826
Epoch 53/100
112/112 [==============================] - 31s 273ms/step - loss: 3.4475 - acc: 0.2045 - val_loss: 3.5716 - val_acc: 0.1901
Epoch 54/100
112/112 [===================

112/112 [==============================] - 31s 281ms/step - loss: 3.6384 - acc: 0.1653 - val_loss: 3.6950 - val_acc: 0.1718
Epoch 46/100
112/112 [==============================] - 31s 277ms/step - loss: 3.6484 - acc: 0.1582 - val_loss: 3.6673 - val_acc: 0.1722
Epoch 47/100
112/112 [==============================] - 31s 276ms/step - loss: 3.6244 - acc: 0.1618 - val_loss: 3.7100 - val_acc: 0.1695
Epoch 48/100
112/112 [==============================] - 31s 275ms/step - loss: 3.6011 - acc: 0.1651 - val_loss: 3.6754 - val_acc: 0.1738
Epoch 49/100
112/112 [==============================] - 31s 275ms/step - loss: 3.6010 - acc: 0.1695 - val_loss: 3.6261 - val_acc: 0.1832
Epoch 50/100
112/112 [==============================] - 31s 274ms/step - loss: 3.5814 - acc: 0.1725 - val_loss: 3.6676 - val_acc: 0.1744
Epoch 51/100
112/112 [==============================] - 31s 275ms/step - loss: 3.5939 - acc: 0.1711 - val_loss: 3.6500 - val_acc: 0.1744
Epoch 52/100
112/112 [==============================] 

In [18]:
X_train, Y_train, X_test, Y_test = data()
val_loss, val_acc = best_model.evaluate(X_test, Y_test);
print("Evalutation of best performing model:")
print("Validation loss: ", val_loss)
print("Validation accuracy: ", val_acc)

Getting data


100%|██████████| 10222/10222 [01:18<00:00, 129.50it/s]


MemoryError: 

In [ ]:
best_model.save(modelPath+modelName);
best_model.summary()

Remember, accuracy is low here because we are not taking advantage of the pre-trained weights as they cannot be downloaded in the kernel. This means we are training the wights from scratch and I we have only run 1 epoch due to the hardware constraints in the kernel.

Next we will make our predictions.

In [ ]:
# preds = model.predict(x_test, verbose=1)

In [ ]:
# sub = pd.DataFrame(preds)
# # Set column names to those generated by the one-hot encoding earlier
# col_names = one_hot.columns.values
# sub.columns = col_names
# # Insert the column id from the sample_submission at the start of the data frame
# sub.insert(0, 'id', df_test['id'])
# sub.head(10)